In [159]:
import numpy as np

In [160]:
import pandas as pd

In [161]:
import nltk

In [162]:
import sklearn

In [163]:

cols = ['polarity','id','date','flag','user','text']
set_encoding = "ISO-8859-1"


In [164]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding=set_encoding, names=cols)
#df.sample(10)

In [165]:
np.sum(df.isnull().any(axis=1))

0

In [166]:
data=df[['text','polarity']]
data.sample(10)

,text,polarity
1361722,GRADUATION TODAY!!!!! SO EXCITING,4
916117,"fun night tonight, and my house looks so purrr...",4
303571,Bubbles died.,0
531916,And now I have a phone call and can't listen t...,0
1182906,@flourishingjudy - now we just need to get #t...,4
1147008,im okay with the fact that im a lesbian at the...,4
1521917,@breathe_in that'd be nice,4
463769,"@Kell_Krushka i know, it makes me sad too.",0
784773,im really tired... no sleep last nite,0
1172508,New Funnelize Firefox themes installed on http...,4


In [167]:
data['polarity'] = data['polarity'].replace(4,1)


/var/folders/wt/vkwnpm091dqdlm0xqwfjxq_80000gn/T/ipykernel_56187/335142359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['polarity'] = data['polarity'].replace(4,1)


In [168]:
data_pos = data[data['polarity'] == 1].iloc[:int(20000)]
data_neg = data[data['polarity'] == 0].iloc[:int(20000)]
dataset = pd.concat([data_pos, data_neg])
dataset['text']=dataset['text'].str.lower()
dataset['text'].tail()
#dataset.sample(20)

19995    not much time off this weekend, work trip to m...
19996                            one more day of holidays 
19997    feeling so down right now .. i hate you damn h...
19998    geez,i hv to read the whole book of personalit...
19999    i threw my sign at donnie and he bent over to ...
Name: text, dtype: object

In [169]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
#print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mikebazarsky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [170]:
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


In [171]:
dataset['text'] = dataset['text'].apply(lambda text: cleaning_stopwords(text))
dataset['text'].tail()

19995    much time weekend, work trip malmï¿½ fri-sat t...
19996                                     one day holidays
19997                   feeling right .. hate damn humprey
19998    geez,i hv read whole book personality types em...
19999     threw sign donnie bent get thingee made sad face
Name: text, dtype: object

In [172]:
import string
english_punctuations = string.punctuation
print(english_punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [173]:
#Removing punctuation
def cleaning_punctuations(text):
    translator = str.maketrans('', '', english_punctuations)
    return text.translate(translator)
dataset['text']= dataset['text'].apply(lambda x: cleaning_punctuations(x))
dataset['text'].tail()

19995    much time weekend work trip malmï¿½ frisat tod...
19996                                     one day holidays
19997                     feeling right  hate damn humprey
19998    geezi hv read whole book personality types emb...
19999     threw sign donnie bent get thingee made sad face
Name: text, dtype: object

In [199]:
#Removing URLs
import re
dataset['text'] = dataset['text'].astype(str)
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)

dataset['text'] = dataset['text'].apply(lambda x: cleaning_URLs(x))
dataset['text'].head()

800000    ['love', 'healthuandpets', 'u', 'guys', 'r', '...
800001    ['im', 'meeting', 'one', 'besties', 'tonight',...
800002    ['darealsunisakim', 'thanks', 'twitter', 'add'...
800003    ['sick', 'really', 'cheap', 'hurts', 'much', '...
800004              ['lovesbrooklyn', 'effect', 'everyone']
Name: text, dtype: object

In [200]:
#Removing numbers
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))
dataset['text'].head()

800000    ['love', 'healthuandpets', 'u', 'guys', 'r', '...
800001    ['im', 'meeting', 'one', 'besties', 'tonight',...
800002    ['darealsunisakim', 'thanks', 'twitter', 'add'...
800003    ['sick', 'really', 'cheap', 'hurts', 'much', '...
800004              ['lovesbrooklyn', 'effect', 'everyone']
Name: text, dtype: object

In [201]:
#Tokenizing
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

In [202]:
tokenizer = RegexpTokenizer(r'\w+')
dataset['text'] = dataset['text'].apply(tokenizer.tokenize)
dataset['text'].head()

800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meeting, one, besties, tonight, cant, wai...
800002    [darealsunisakim, thanks, twitter, add, sunisa...
800003    [sick, really, cheap, hurts, much, eat, real, ...
800004                    [lovesbrooklyn, effect, everyone]
Name: text, dtype: object

In [206]:
#Stemming
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
dataset['text']= dataset['text'].apply(lambda x: stemming_on_text(x))
dataset['text'].head()


800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meeting, one, besties, tonight, cant, wai...
800002    [darealsunisakim, thanks, twitter, add, sunisa...
800003    [sick, really, cheap, hurts, much, eat, real, ...
800004                    [lovesbrooklyn, effect, everyone]
Name: text, dtype: object

In [205]:
#Lemmatizer
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
dataset['text'] = dataset['text'].apply(lambda x: lemmatizer_on_text(x))
dataset['text'].head()

800000             [love, healthuandpets, u, guys, r, best]
800001    [im, meeting, one, besties, tonight, cant, wai...
800002    [darealsunisakim, thanks, twitter, add, sunisa...
800003    [sick, really, cheap, hurts, much, eat, real, ...
800004                    [lovesbrooklyn, effect, everyone]
Name: text, dtype: object

In [204]:
#Separating input feature and label
X=data.text
y=data.polarity

In [207]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state =26105111)
